<font face="Times New Roman">
<div dir=ltr align=center>
<font color=3C99D size=6>
    Blocked Reactions in Metabolic Models

In this notebook, we want to find blocked reactions using classic and modern approaches to solve an optimization problem.

## 1.1

In [ ]:
using COBREXA

e_coli_core = load_model(StandardModel, "./Models/e_coli_core.json")
iAB_RBC_283 = load_model(StandardModel, "./Models/iAB_RBC_283.json")
iNF517 = load_model(StandardModel, "./Models/iNF517.json")
iNJ661 = load_model(StandardModel, "./Models/iNJ661.json")
Recon3D = load_model(StandardModel, "./Models/Recon3D.json")

models = Dict("e_coli_core" => e_coli_core,
    "iAB_RBC_283" => iAB_RBC_283,
    "iNF517" => iNF517,
    "iNJ661" => iNJ661,
    "Recon3D" => Recon3D)

In [ ]:
mutable struct GEMObj
    S
    Metabolites
    Reactions
    Genes
    m
    n
    lb
    ub
end

In [ ]:
myModels = Dict()
for (k, v) in models
    myModels[k] = GEMObj(stoichiometry(v),
        metabolites(v),
        reactions(v),
        genes(v),
        n_metabolites(v),
        n_reactions(v),
        lower_bounds(v),
        upper_bounds(v))
end

### Printing Models

In [ ]:
for (k, e) in myModels
    println("S = ", e.S)
    println("Metabolites = ", e.Metabolites)
    println("Reactions = ", e.Reactions)
    println("Genes = ", e.Genes)
    println("Met count = ", e.m)
    println("Rxn count = ", e.n)
    println("Lower Bounds = ", e.lb)
    println("Upper Bounds = ", e.ub)
    println()
end

## 1.2

In [ ]:
Reversible = Dict()
Irreversible = Dict()
for (k, v) in myModels
    for (i, e) in enumerate(v.lb)
        if e < -1e-8
            if k in keys(Reversible)
                Reversible[k] = push!(Reversible[k], i)
            else
                Reversible[k] = [i]
            end
        else
            if k in keys(Irreversible)
                Irreversible[k] = push!(Irreversible[k], i)
            else
                Irreversible[k] = [i]
            end
        end 
    end
end

## 1.3

In [ ]:
for (k, e) in myModels
    for i in Irreversible[k]
        e.lb[i] = 0
        e.ub[i] = 1000000
    end
    
    for i in Reversible[k]
        e.lb[i] = -1000000
        e.ub[i] = 1000000
    end
end

## 2.1

In classic method, we want to solve two below optimizations to find blocked reactions:
$$\max v_i \hspace{150pt} \min v_i$$
$$ \hspace{15pt} s.t. \hspace{5pt} v \in C \hspace{135pt} s.t. \hspace{5pt} v \in C $$
$$ \hspace{40pt} v_i \leq 1 \hspace{155pt} v_i \geq -1 $$

In [ ]:
using JuMP
import GLPK

function ClassicMethod(k, e, IBR, RBR)
    
    for i in Irreversible[k]
        optModel = Model(GLPK.Optimizer)

        @variable(optModel, e.ub[j] >= v[j=1:e.n] >= e.lb[j])
        
        # V(Irreversible) >= 0
        for j in Irreversible[k]
            @constraint(optModel, v[j] >= 0)
        end
        
        # Sv = 0
        @constraint(optModel, e.S * v .== 0)
        
        # vi <= 1
        @constraint(optModel, v[i] <= 1)
        
        @objective(optModel, Max, v[i])
        
        optimize!(optModel)
        
        if isapprox(objective_value(optModel), 0; atol=1e-8)
            if k in keys(IBR)
                IBR[k] = push!(IBR[k], i)
            else
                IBR[k] = [i]
            end
        end
    end
    
    for i in Reversible[k]
        optModel1 = Model(GLPK.Optimizer)

        @variable(optModel1, e.ub[j] >= v[j=1:e.n] >= e.lb[j])

        for j in Irreversible[k]
            @constraint(optModel1, v[j] >= 0)
        end
        @constraint(optModel1, e.S * v .== 0)
        @constraint(optModel1, v[i] <= 1)
        
        @objective(optModel1, Max, v[i])
        
        optimize!(optModel1)
        
        if isapprox(objective_value(optModel1), 0; atol=1e-8)
            optModel2 = Model(GLPK.Optimizer)

            @variable(optModel2, e.ub[j] >= v[j=1:e.n] >= e.lb[j])
            
            for j in Irreversible[k]
                @constraint(optModel2, v[j] >= 0)
            end
            @constraint(optModel2, e.S * v .== 0)
            @constraint(optModel2, v[i] >= -1)
            
            @objective(optModel2, Min, v[i])
            
            optimize!(optModel2)
            
            if isapprox(objective_value(optModel2), 0; atol=1e-8)
                if k in keys(RBR)
                    RBR[k] = push!(RBR[k], i)
                else
                    RBR[k] = [i]
                end
            end
        end
    end
end

In [ ]:
IrrBlockedReactions = Dict()
RBlockedReactions = Dict()

for (k, e) in myModels
    
    if k == "Recon3D"
        continue
    end
    
    println(k)
    @time ClassicMethod(k, e, IrrBlockedReactions, RBlockedReactions)
    
end

In [ ]:
for k in keys(myModels)
    if k in keys(IrrBlockedReactions) && k in keys(RBlockedReactions)
        println(k, " Blocked Irreversible Reaciotns: ", length(IrrBlockedReactions[k]))
        println(k, " Blocked Reversible Reaciotns: ", length(RBlockedReactions[k]))
        println(k, " Total Blocked Reaciotns: ", length(IrrBlockedReactions[k]) + length(RBlockedReactions[k]))
        println()
    elseif k in keys(IrrBlockedReactions)
        println(k, " Blocked Irreversible Reaciotns: ", length(IrrBlockedReactions[k]))
        println(k, " Total Blocked Reaciotns: ", length(IrrBlockedReactions[k]))
        println()
    elseif k in keys(RBlockedReactions)
        println(k, " Blocked Reversible Reaciotns: ", length(RBlockedReactions[k]))
        println(k, " Total Blocked Reaciotns: ", length(RBlockedReactions[k]))
        println()
    elseif k == "Recon3D"
        println(k, " Not Calculated")
        println()
    else
        println(k, " Total Blocked Reaciotns: 0")
        println()
    end
end

## 2.2

In modern approach we solve a single optimization.
$$ \max \hspace{5pt} 1^T \min(v_I, 1)$$
$$ s.t. \hspace{10pt} v \in C$$

In [ ]:
using JuMP
import GLPK

function ModernMethod(k, e, MIRBR)
    optModel = Model(GLPK.Optimizer)
    
    n_irr_r = length(Irreversible[k])
    
    @variable(optModel, v[1:e.n])
    @variable(optModel, 1 >= u[1:n_irr_r] >= 0)

    for (i, j) in enumerate(Irreversible[k])
        @constraint(optModel, v[j] >= u[i])
    end
    @constraint(optModel, e.S * v .== 0)
    
    @objective(optModel, Max, transpose(ones(n_irr_r)) * u)

    optimize!(optModel)

    println(" Irreversible Blocked Reactions: ", n_irr_r - objective_value(optModel))
    for (i, j) in enumerate(value.(v))
        if i in Irreversible[k]
            if isapprox(j, 0; atol=1e-8)
                if k in keys(MIRBR)
                    MIRBR[k] = push!(MIRBR[k], i)
                else
                    MIRBR[k] = [i]
                end
            end
        end
    end
end

In [ ]:
modernIrrBlockedReactions = Dict()

for (k, e) in myModels
    print(k)
    @time ModernMethod(k, e, modernIrrBlockedReactions)
end

In [ ]:
for k in keys(myModels)
    if k in keys(modernIrrBlockedReactions)
        println(k, " Blocked Irreversible Reaciotns: ", length(modernIrrBlockedReactions[k]))
        println()
    else
        println(k, " Blocked Irreversible Reaciotns: 0")
        println()
    end
end

## 3.3

In [ ]:
cobra_iNF517 = ["ACMANApts", "ALOX", "AMALT1", "AMALT2", "AMALT3", "AMALT4", "ACGAMT", "ASNt2r", "ACOATA", "BIOMASS_LLA_noATPnoH", "ACODA_1", "ACONT", "2MPAt6", "ACKILE", "3MBALt", "3MBAt6", "BZALt", "ACKLEU", "2S6HCCi", "ACKVAL", "CITt2r", "AB6PGH", "CITt3", "ACPS2", "ACPSc", "ADK2", "CSND", "DRIBt2", "CYTDK1", "AEPabc", "CYTDK2", "EX_2aeppn_e", "AGDC", "EX_2mpa_e", "EX_3mba_e", "EX_3mbal_e", "AHMMPS_1", "ASPCT", "EX_acgala_e", "EX_acgam_e", "EX_acmana_e", "ASPR", "ASPTA2", "ALCD19", "BG_CELLB", "CARORSr", "CGPT", "CHITN1", "EX_asn__L_e", "AMANAPEr", "CHLabc", "AMMQT8_2", "CPPPGO", "EX_bzal_e", "CMLDC", "CRPT_25", "CS", "EX_cellb_e", "EX_chol_e", "EX_cit_e", "APRAUR", "DHPPDA", "CYTD", "CYTB_B2", "DGNSK", "DINSK", "DB4PS", "DHNAOT_1", "EX_drib_e", "EX_fe2_e", "DHORDi", "DHORTS", "EX_fe3_e", "FE2abc", "DRBK", "GLYBabc", "EX_fru_e", "FE3abc", "EX_gal_e", "EX_galt_e", "DXPS", "EDA", "FLVR_1", "FMETDF", "FMETTRS_1", "FMNAT", "FTHFCL_1", "G3PCT", "EX_glcn__D_e", "EX_gln__L_e", "G3PD4", "ILEt2r_copy1", "EX_glyb_e", "GALK2", "GALM", "LCTSt", "GALT", "EX_glyc3p_e", "EX_glyc_e", "GALt2", "GTPCII", "GTHPi", "GTHOr", "GPDDA4", "GLCNt2ir", "EX_hxan_e", "GTPDPK", "HXANt2r", "GLNabc", "GUI1", "GUI2", "HARAAT", "GLYC3Pt6", "GLYCt", "EX_lcts_e", "HIBD", "GNKr", "GPDDA1", "GPDDA2", "EX_mal__L_e", "LEUt2r_copy1", "EX_malt_e", "HSST", "EX_man_e", "EX_met__D_e", "ICDHyr", "ICHORS", "EX_mn2_e", "EX_mnl_e", "EX_nmn_e", "EX_o2_e", "MTAN", "LACZ", "IPPMIa", "IPPMIb", "ARAT", "O2t", "M1PD", "MALLAC", "MALTabc", "MALTATr", "MALt2r", "MAN6PI", "MANAO", "PHPYROX", "PROTS_LLA", "PROTS_LLA_v2", "EX_ppi_e", "PFK_2", "MI1PP", "MNNH", "EX_ptrc_e", "MNabc", "MHPGLUT", "NADK", "NADH4", "OHPBAT", "EX_rib__D_e", "NMNP", "NMNAT", "EX_ribflv_e", "EX_sbt__D_e", "NPHS", "EX_ser__D_e", "NOX", "NOX2", "NADHPO", "EX_so4_e", "EX_spmd_e", "RIBFLVt2", "RIBt2", "EX_sucr_e", "SPP", "SBTpts", "PPIabc", "EX_tre_e", "POR", "DSERt2", "EX_trp__L_e", "RBFK", "SUCR", "PTAILE", "PTALEU", "PTAVAL", "PREN", "PTRCabc", "PUNP3", "PUNP4", "PYNP4", "EX_zn2_e", "PYROX_1", "RBFSa", "RBFSb", "RIBabc", "S7PI_1", "TRPt2r", "VALt2r_copy1", "SHSL1", "URIK3", "THDPS", "SO4t2", "SPODM", "THFGLUS", "SPMDAT1", "SPMDAT2", "SUCBZL", "SUCBZS", "S6PG", "TMDPP", "OIVD1r", "OIVD2", "OIVD3", "PDH", "SPMDabc", "TKT1", "PUNP7", "ACGApts", "TRPS2", "ITCY", "TRE6PH", "GALTpts", "UAAGDS", "TREpts", "GPDDA3", "UDCPKr", "MNLpts", "UDPGALM", "UDPGD", "GPDDA5", "ACGALpts", "XYLI1", "ZNabc", "ASPTA4", "MANpts", "CELBpts", "DHORD6", "FRUK", "FRUpts", "METDabc", "BGLA1", "SUCpts"]
cobra_iNJ661 = ["THZSN", "R05224", "R05219", "TARTRt7", "R05217", "TARTD", "QULNS", "THZPSN", "SUCBZS", "PYDXO", "PYDXNO", "PYAM5PO", "THMDP", "THFGLUS", "SUCBZL", "PUNP4", "PSSA180", "SPMS", "PSD180", "PHTHCLS", "SHCHF3", "PRDX", "SHCHF2", "SHCHF", "SHCHD3", "PGLYCP", "PPTGS", "PPPGO", "SHCHD2", "SHCHCS2", "SHCHCS", "PERD", "PDX5POi", "PPBNGS", "PNCDC", "PMPK", "SERD_D", "PMDPHT", "PC8XM", "PC6YM", "SELCYSLY", "PC6AR", "PC20M", "PC17M", "PC11M", "MPML", "MOTS", "MOTH", "MNDNS2", "MNDNS1", "RZ5PP", "OXGDC2", "MTAN", "MSHS", "MSHOXH", "MSHAMID", "MMSAD2", "MMSAD1", "MI4PP", "MI3PP", "RIBabc", "RBK_L1", "MHPGLUT", "MGt5", "METabc", "TTDCAt2", "RBFSb", "TSULabc", "RBFSa", "MEOHt2", "TRIATS", "TREabc", "MDFDH", "TMPPP", "TMPK", "TMN", "TMHAS6", "MCBTS2", "TMHAS5", "TMDPP", "YUMPS", "XYLabc", "MALTabc", "XYLK", "MALTHPabc", "UREA", "UPPRT", "UPPDC2", "UPPDC1", "UPP3S", "UPP3MT", "LPLIPAL2E180", "LCTSabc", "LACZ", "UDPGD", "GLYCTO1", "IGAMD", "GLYCK", "GLYC3Pabc", "GLYBabc", "ICHORS", "IACGAMS", "GLUTRR", "HSERTA", "GLNSP3", "EX_lcts_e", "EX_k_e", "GLNSP2", "GLNSP1", "HPPK2", "EX_hdca_e", "HPPK", "HOPNTAL", "HMPK4", "GCC2", "GALt2", "EX_gthrd_e", "EX_glyc3p_e", "EX_glyb_e", "EX_gal_e", "HMPK3", "EX_fe2_e", "HMPK2", "EX_cu2_e", "EX_crn_e", "G1SAT", "HMPK1", "EX_cobalt2_e", "EX_co_e", "EX_cl_e", "HMBS", "EX_chol_e", "EX_ca2_e", "EX_bmn_e", "EX_atp_e", "EX_arab__L_e", "EX_arab__D_e", "HDCAt2", "FRDO", "EX_met__L_e", "EX_meoh_e", "EX_malthp_e", "EX_malt_e", "OHPBAT", "OCTDPS", "NPHS", "OCDCAt2", "NOt", "NO3t7", "NO3R2", "NO3R1", "NO", "NTRIRy", "NTRIRx", "NTRIR2x", "NNDPR", "NNDMBRT", "NNATr", "NNAM", "NTPTP1", "NH4OHDs", "NTD8", "NTD6", "NARK", "NAPRT", "GTPDPK", "GTPCII2", "GTPCII", "GTPCI", "NADS2", "NADK", "EX_acysbmn_e", "GTHRDt2", "ENTCS", "E4PD", "MYCFLUX5", "MYCFLUX4", "MYCFLUX3", "DURIPP", "MYCFLUX2", "MYCFLUX1", "DSERt2", "DRPA", "DNTPPA", "DNMPPA", "DM_atp_c", "Clt", "DHPS2", "DHPPDA2", "DHPPDA", "DHNPA2r", "GLYCTO4", "GLYCTO3", "DHNPA", "GLYCTO2", "CYRDAR", "DHNAOT2", "DHNAOT", "DHFS", "CYRDAAT", "DGC3D", "CRNabc", "CPPPGO", "CPC8MM", "CPC6R", "CPC6MT", "CPC4MT", "CPC3MT", "CPC2MT", "COt", "COCHL", "DCPDP", "DBTS", "DB4PS", "COBALTt5", "CMLDC", "CMCBTFU", "ASPO5", "CITL_copy2", "CIGAMS", "CHOLD", "CHLabc", "ADOCBLS", "ADOCBIK", "CBLAT", "CBIAT", "BTS3r", "BTS2", "BMNt", "BMNMSHS", "FOLR2", "FOLD3", "FMNAT", "BETALDHx", "FMETTRS", "FETRANS", "BACCL", "FE3abc", "FE3Ri", "FE2abc", "ARGDC", "FCLT2", "FCLT", "FCLPA", "ARB_Dabc", "FASm3001", "FAH4", "ARAI", "EX_xyl__D_e", "APRAUR", "EX_ttdca_e", "EX_tsul_e", "EX_tre_e", "AOXSr", "EX_tartr__L_e", "EX_ser__D_e", "ACPS1", "EX_rib__D_e", "AMMQT8_2", "EX_ocdca_e", "AMMQT8", "EX_no_e", "EX_no3_e", "EX_no2_e", "AMMQT6", "EX_na1_e", "EX_mg2_e", "AMID4", "ADMDC", "ACBIPGT", "AMAOTr", "ADCYRS", "ADCS", "ADCPS2", "ADCPS1", "ADCL", "5DGLCNR", "ALDD1", "4HTHRS", "ACYSBMNt", "ALATRS", "AKGDb", "AHMMPS", "AHCYSNS", "AGMT"]
cobra_e_coli_core = ["EX_fru_e", "EX_fum_e", "EX_gln__L_e", "EX_mal__L_e", "FRUpts2", "FUMt2_2", "GLNabc", "MALt2_2"]
cobra_iAB_RBC_283 = ["EX_ca2_e", "EX_cl_e", "EX_fe2_e", "EX_k_e", "EX_na1_e", "EX_phe__L_e", "AP4AH1", "EX_ribflv_e", "FE2t", "GALOR", "ICDHyr", "MDH", "PHETA1", "PHEtec", "PMANM", "UMPK"]

BR = Dict()

for (k, e) in myModels
    if k in keys(IrrBlockedReactions)
        for i in IrrBlockedReactions[k]
            if k in keys(BR)
                BR[k] = push!(BR[k], e.Reactions[i])
            else
                BR[k] = [e.Reactions[i]]
            end
        end
    end
    
    if k in keys(RBlockedReactions)
        for i in RBlockedReactions[k]
            if k in keys(BR)
                BR[k] = push!(BR[k], e.Reactions[i])
            else
                BR[k] = [e.Reactions[i]]
            end
        end
    end
end

classic_cobra_iNF517 = intersect(BR["iNF517"], Set(cobra_iNF517))
classic_cobra_iNJ616 = intersect(BR["iNJ661"], Set(cobra_iNJ661))
classic_cobra_e_coli_core = intersect(BR["e_coli_core"], Set(cobra_e_coli_core))
classic_cobra_iAB_RBC_283 = intersect(BR["iAB_RBC_283"], Set(cobra_iAB_RBC_283))

println(length(classic_cobra_iNF517))
println(length(classic_cobra_iNJ616))
println(length(classic_cobra_e_coli_core))
println(length(classic_cobra_iAB_RBC_283))

## 3.1
#### Updating Models to Consisnent with Flux

In [ ]:
for (k, e) in myModels

    BRxnName = []
    
    if k in keys(IrrBlockedReactions)
        for i in IrrBlockedReactions[k]
            push!(BRxnName, e.Reactions[i])
        end
    end
    
    if k in keys(RBlockedReactions)
        for i in RBlockedReactions[k]
            push!(BRxnName, e.Reactions[i])
        end
    end

    if k in keys(IrrBlockedReactions) && k in keys(RBlockedReactions)
        e.S = e.S[:, setdiff(1:e.n, sort(append!(IrrBlockedReactions[k], RBlockedReactions[k])))]
        e.lb = deleteat!(e.lb, unique(sort(append!(IrrBlockedReactions[k], RBlockedReactions[k]))))
        e.ub = deleteat!(e.ub, unique(sort(append!(IrrBlockedReactions[k], RBlockedReactions[k]))))
    elseif k in keys(IrrBlockedReactions)
        e.S = e.S[:, setdiff(1:e.n, sort(IrrBlockedReactions[k]))]
        e.lb = deleteat!(e.lb, sort(IrrBlockedReactions[k]))
        e.ub = deleteat!(e.ub, sort(IrrBlockedReactions[k]))
    elseif k in keys(RBlockedReactions)
        e.S = e.S[:, setdiff(1:e.n, sort(RBlockedReactions[k]))]
        e.lb = deleteat!(e.lb, sort(RBlockedReactions[k]))
        e.ub = deleteat!(e.ub, sort(RBlockedReactions[k]))
    end
    
    e.Reactions = setdiff(e.Reactions, Set(BRxnName))
    e.n -= length(BRxnName)
    
end